# Apprentissage par renforcement

## Introduction

L’apprentissage par renforcement est une discipline de l’intelligence artificielle, visant à faire évoluer un agent dans son environnement afin qu’il apprenne par lui-même à le maîtriser. Cette discipline est inspirée des neurosciences et de la psychologie expérimentale que l’on a adaptée à l’informatique, au début des années 1980. En effet les neurosciences ont mis à jour le renforcement des poids synaptiques des transmissions neuronales lorsque son activité devenait plus importante. D’un autre côté, l’expérience anthropologique a permis de découvrir des modèles de conditionnement chez les animaux, reposant sur la satisfaction. C’est la base du dressage : on stimule une douleur en cas d’échec et un plaisir en cas de succès. 

![Schéma_renforcement](https://image.noelshack.com/fichiers/2019/23/2/1559676471-whatsapp-image-2019-04-17-at-10-21-35.jpeg)

C’est sur ces principes que se base en partie l’apprentissage par renforcement. Au début, l’algorithme est naïf et sans expérience. Il va évoluer de manière aléatoire dans son environnement. Ses évolutions sont évaluées par une fonction. L’algorithme est alors puni ou récompensé selon son action, et apprend. A l’image de l’expérience d’un rat dans une cage qui se voit assigné une croquette sécrétant de la dopamine ou un choc électrique stimulant la douleur, selon le levier qu’il presse.

C’est ainsi que les suites d’actions menant à la somme des récompenses la plus élevée sont retenues et exploitées, tout comme les poids synaptiques s’accentuent lors d’une activité importante entre deux neurones. Les chaines d’actions menant aux meilleurs résultats sont priorisées par la suite.

Tout au long de ce cours, nous utiliserons un exemple pour illustrer les différents concepts abordés: **le jeu du labyrinthe**.

![Matrice Jeu ](https://image.noelshack.com/fichiers/2019/23/2/1559676166-grille-labyrinthe.png)

Le principe est simple:
* la case bleue représente la case départ
* le joueur évolue à l'aveugle dans la grille
* le joueur perd s'il arrive dans la case rouge
* le joueur gagne s'il arrive dans la case jaune

Le but est d'arriver à entraîner une IA pour trouver le chemin idéal dans le labyrinthe.


## 1 - Processus Markoviens

Le formalisme des processus markoviens fournit un cadre précis et adapté à l'abstraction des techniques d'apprentissage par renforcement. N'hésitez pas à revenir à ce paragraphe régulièrement si vous n'êtes pas sûr de la définition d'un terme par exemple.

###Formalisme
**Niveau 1 :**

Dans l’apprentissage par renforcement, l’environnement est modélisé par un ensemble d’états. L’agent peut passer d’un état à un autre en effectuant une action. L’environnement renforce l’agent après chaque action en lui renvoyant une récompense positive ou négative. C’est ce qui permet « d’éduquer » l’agent. Le but de l’agent est de maximiser cette récompense.


* **AGENT** : c’est l’agent qui fait des actions et qui explore son environnement ; par exemple un robot qui se déplace dans notre labyrinthe, un personnage de jeu vidéo …

![agent](https://image.noelshack.com/fichiers/2019/23/2/1559676615-agent.png)
* **ETAT $s \in S$** : $S$ est l’ensemble des états du système. Un état $s$ correspond à la position où se trouve l’agent, ainsi qu'éventuellement d'autres caractéristiques pertinentes (barre de vie, munitions pour un jeu vidéo par exemple)

![etat](https://image.noelshack.com/fichiers/2019/23/2/1559677041-etat.png)

* **ACTION $a \in A$** : $A$ est l’ensemble de toutes les actions que l’agent peut faire. $a$ est une action de l’ensemble $A$, elle peut être par exemple pour le robot: se déplacer à droite, se déplacer à gauche …

![action](https://image.noelshack.com/fichiers/2019/23/2/1559676990-action.png)

* **ENVIRONNEMENT** : c’est le cadre d’action de l’agent, le labyrinthe dans notre exemple. L’environnement prend l’état de l’agent et une action pour retourner une récompense et un nouveau état.

![environnement](https://image.noelshack.com/fichiers/2019/23/2/1559677235-environnement.png)
* **RECOMPENSE $r \in R$** : $R$ est l'ensemble des récompenses possibles. Une récompense $r$ correspond à l’évaluation des actions de l’agent. Dans notre exemple, le robot est récompensé (+100) s'il arrive dans la case jaune, et est pénalisé s'il arrive dans la case rouge (-100).

![recompense](https://image.noelshack.com/fichiers/2019/23/2/1559677462-recompense.png)
* **POLITIQUE $\pi$** : (avant tout un francissisme de "policy") c’est la stratégie que l’agent utilise pour déterminer ses actions futures selon son expérience passée et son état présent et qui garantie la meilleure récompense. $\pi (s,a)$ est la probabilité que l'agent dans l'état s effectue l'action a.


Nous utilisons pour cela un outil de représentation qui s’appelle processus de Markov à temps discret ou bien chaine de Markov. Il permet de modéliser les différents états d’un système, accompagné des transitions possibles entre ces états.

Exemple :

![MDP](https://image.noelshack.com/fichiers/2019/23/2/1559677901-mdp.png)

Dans une chaine de Markov chaque sommet représente un état et chaque arrête une transition possible entre les deux états. Le poids de l’arrête représente la probabilité que cette transition ait lieu. Dans le cadre de l’apprentissage par renforcement on associe également une récompense à chaque action effectuée dans un certain état. C’est ce qui permettra « d’éduquer » l’algorithme.

**Niveau 2 :**

Définissons ce processus de Markov de manière plus formelle :
* $S$ est un ensemble fini d’états discrets notés $s$ (ce sont les sommets du graphe).
* $A$ est un ensemble fini d’actions $a$ (c’est l’ensemble des arcs orientés).
* $T : S × A × S → [0 ; 1]$ est une fonction de transition associant la probabilité de transition de l’état s vers un autre état par une action de A.
* $R : S × A → R$,  représente la récompense d’effectuer l’action a dans l’état s, c’est une valeur réelle symbolisant une récompense ou une punition. (Ex : valeur positive ou négative)

Dans le cadre de l’apprentissage par renforcement, les probabilités de transition ne sont pas connues de l'agent qui apprend.

Si l’on revient a notre exemple précédent, nous aurons :

 * $S=${ 1, 2, 3 ,4 ,5, Etat Terminal }.
 * $A=$ {Haut, Droite, Bas, Gauche}

Dans l’état 1, les seules actions possibles sont {Haut , Droite} et on pourrait avoir :
 * $T(1, \text{Haut})=0.5$ 
 * $T(1, \text{Droite})=0.5$ 

Nécessairement $T(1, \text{Gauche})=0$ et $T(1, \text{Bas})=0$. Les deux actions Haut et Droite sont équiprobables.
   
Exemple d’une fonction de récompense :  
$R(1, \text{Haut})= -10$ 	(c’est le puits) \
$R(3,\text{Droite})= +10 $ (c’est la sortie) \
$R(1,\text{Droite})= -1 $  (c’est une case sans effet, on lui associe neanmoins une récompense négative afin de minimiser le nombre de coups)

![MDP2](https://image.noelshack.com/fichiers/2019/23/3/1559760644-mdp2.png)

Les fonctions de transition $T$ et de récompense $R$ permettent de définir complétement le problème et font partie de la donnée de l'environnement. Toutefois cette représentation ne permet pas de choisir l’action $a$ qui maximisera la récompense de l’agent lorsqu’il est dans l’état $s$. 
On utilise pour ça une politique $\pi$ et une fonction de valeur $V$.



###Principe formel de l'apprentissage par renforcement
**Niveau 1**

L’apprentissage par renforcement permet d’évaluer une politique $\pi$, c’est-à-dire une stratégie de prise de décision. Pour cela, on utilise une fonction de valeur $V$. Cette fonction permet de "juger" chaque état, en leur associant une valeur. Plus cette valeur est grande, plus l'agent a intérêt à aller dans cet état. À l'inverse, plus cette valeur est faible, plus il a intérêt à l'éviter. Pendant l'apprentissage, l'agent va modifier la fonction de valeur pour la rendre optimale.

Les étapes sont les suivantes :

* On fixe la politique $\pi$ à évaluer
* On initialise la fonction de valeur $V^{\pi}$
* L’agent évolue dans son environnement. A chaque étape :
	- l’agent est dans un état k
	- une action est déterminée par la politique choisie
	- l’agent arrive dans l’état k+1 et reçoit la récompense relative à cet état
	- on actualise la fonction de valeur $V^{\pi}$
* On répète l’étape précédente jusqu’à obtenir une fonction de valeur $V^{\pi}$ satisfaisante (ex : qui ne change plus trop malgré de nouvelles parties, c'est-à-dire qu'elle a convergé, même si elle est mauvaise, auquel cas on réfléchit comment mieux faire apprendre)

Le concept de fonction de valeur est traité plus en détail dans la partie 2.

**Niveau 2**

Dans le cas d'un problème décisionnel dit à horizon temporel fini n, c'est-à-dire quand l'interaction entre l'agent et l'environnement se termine au bout de $n<\infty$ itérations, la fonction de valeur $V$ évaluant la politique $\pi$ est définie ainsi:

$$ V^{\pi}(s) = \mathbb{E}^{\pi} \begin{bmatrix} \sum_{t=0}^{n-1} r(s_{t},a_{t})|s_{0}=s\end{bmatrix}  $$

avec $ \mathbb{E}^{\pi}$ l'espérance lorsque l'agent suit la politique $\pi$.

Pendant l'apprentissage, l'objectif de l'agent est de déterminer une politique qui lui permette de gagner la plus grande récompense possible en maximisant la fonction de valeur. 

On dit qu’une politique $\pi^{*} $ est *optimale* si elle vérifie:
$$ \forall \pi, \forall x \in X, V^{\pi}(x) \leq V^{\pi^{*}}(x)$$

On peut montrer que comme les états et les actions sont en nombre fini, il existe bien  une telle politique optimale déterministe. L'objectif est donc de trouver pendant l'apprentissage une politique $\pi$ se rapprochant le plus possible de cette politique optimale $\pi^{*}$.

Cependant, la fonction de valeur $V$ n'est pas forcément bien définie de la manière précédente dans le cas d'un problème décissionnel à horizon temporel infini, comme c'est le cas pour le jeu du labyrinthe (l'agent peut très bien se déplacer sur la grille indéfiniment sans rencontrer un état terminal, la cause jaune ou la case rouge). En effet, la somme $\sum_{t=0}^{\infty} r(s_{t},a_{t})$ peut très bien diverger. Pour cela, on introduit un *facteur de pondération $\gamma \in [0;1[$*. Comme la fonction de récompense $r$ est bornée, on a bien cette fois ci la convergence de la somme $\sum_{t=0}^{\infty} \gamma^{t}r(s_{t},a_{t})$. On définie alors la fonction de valeur ainsi:

$$ V_{\gamma}^{\pi}(s) = \mathbb{E}^{\pi} \begin{bmatrix} \sum_{t=0}^{\infty}\gamma^{t} r(s_{t},a_{t})|s_{0}=s\end{bmatrix}  $$


## 2- TD-Learning, Q Learning

### Fonction de valeur

**Niveau 1**

Pour que notre agent (donc en pratique : notre algorithme) apprenne, il faut le doter d'une mémoire de manière à ce qu'il puisse se souvenir de toutes les récompenses ou punitions qu'il a reçu durant ses précédentes parties. C'est là qu'entre en jeu la fonction de valeur : elle permet à l'agent d'avoir accès rapidement à une information sur son environnement qui a été créée par son expérience seule. Comme il la recalcule fréquemment (en général et dans le cadre de ce notebook, après chaque action) elle représente assez fidèlement (pour qui sait l'interpréter) les connaissances que l'agent a appris de son environnement.
Mais qu'est-ce qu'une fonction de valeur ? 

En pratique, comme on le verra, c'est (dans le cas le plus simple) une fonction qui associe à chaque état une valeur qui représente l'intérêt de cet état pour l'agent. Ce n'est pas tant la récompense immédiate donnée par cet état qui est prise en compte, mais plutôt les récompenses futures que l'état pourra amener. Ainsi lorsque notre agent a bien appris après plusieurs entraînements, il se laisse guider par sa fonction de valeur qui le mène vers un chemin optimal pour lui (mais parfois, malgré un entraînement long, l'agent peut rater des choses intéressantes, surtout si l'environnement est grand).

Encore plus en pratique, si notre jeu n'est pas trop gros (nombre d'états réduit, par exemple : ni un jeu vidéo ni une partie d'échec), la fonction de valeur est juste une double liste : une liste d'états, et la liste des valeurs de ces états.

Exemple :

|Etat #1	| Etat #2	| Etat #3	| Etat #4 |
|-|-|-|-|
|V(1) | V(2)	| V(3)	| V(4) |


On peut appliquer cette méthode à notre labyrinthe:

![gif_V_function](https://media.giphy.com/media/Kx7LllAPfimFar0u8A/giphy.gif)

Ici, les numéros dans chaque case indiquent la fonction de valeur appliquée à l'état correspondant à la case. Les flèches pointent la case adjacente avec la valeur la plus élevée.


**Niveau 2**

Le TD-Learning (Temporal Difference Learning) et le Q-Learning sont deux algorithmes d'apprentissage par renforcement qui fonctionnent avec une fonction de valeur qui caractérise la récompense maximale que le joueur peut obtenir selon ses actions. 
Plus précisément, d’un côté, le TD-learning fonctionne avec une fonction de valeur V qui attribue un nombre à chaque état. Plus ce nombre est grand, plus l'état est intéressant. Ainsi après avoir précisé sa propre fonction de valeur (qu'il initialise à 0 partout, ou aléatoirement) en jouant de multiples parties, l'agent tend vers une fonction de valeur unique. Lorsqu'il détermine qu'il a suffisamment bien précisé sa fonction de valeur (tellement bien que malgré les autres parties d'entraînement, elle ne change quasiment plus), il a fini l'entraînement et est capable de joueur optimalement (selon lui). Jouer optimalement consiste à partir de chaque état à aller vers l'état avec la plus grande valeur, dans l'espoir d'obtenir les plus grande récompenses (la fonction de valeur de chaque état ne correspond pas à la récompense de cet état, mais en donne une estimation et surtout donne une estimation des récompenses futures possibles après cet état)
De l'autre côté, le Q-learning utilise une fonction similaire Q, qui attribue un nombre à chaque couple $(\text{etat}, \text{action})$ : cela signifie que chaque action est notée individuellement depuis chaque état, selon le même principe que précédemment. Donc jouer optimalement avec une fonction Q bien déterminée après un entraînement consiste à : depuis chaque état, jouer l'action avec la plus grosse valeur (Q-learning) au lieu de : depuis chaque état, jouer l'action permettant d'aller dans l'état de plus grosse valeur. La différence est subtile, mais elle se traduit par des différences de comportement pouvant être embêtantes sur des jeux spécifiques.

*Exemple :  Dans les jeux non-déterministes (jeu où une certaine action depuis un certain état ne donne pas toujours le même résultat), il est plus intéressant de savoir quel est l'intérêt de jouer une certaine action plutôt que de savoir vers quel état aller, sachant qu'on ne pourra pas y aller de manière certaine.* 

On voit donc que le fait de noter les couples $(\text{etat}, \text{action})$ plutôt que simplement les états est une méthode plus générale : pour ces raisons, nous allons nous concentrer à présent sur le Q-Learning dans la suite du cours (sauf pour le niveau 1 suivant car son cadre correspond à celui du TD(0), un certain type de TD-Learning (le plus simple) )

Enfin, il existe un autre type d'algorithme (en fait, beaucoup d'autres) appelé SARSA et très proche du QLearning, qui différe légèrement dans sa manière d'actualiser la fonction de valeur et qui a l'avantage pratique de faire joueur de manière plus sûre quoique parfois moins optimale par rapport au QLearning classique (quelle que soit la stratégie associée, voir partie suivante).
Voir ce lien pour une comparaison intéressante : http://www-igm.univ-mlv.fr/~dr/XPOSE2014/Machin_Learning/A_Exemple.html

### Comment entraîner cette fonction de valeur ?

**Niveau 1**

Reprenons le cas d'un jeu pas trop gros, et pour ceux qui ont lu le niveau 2 précédent, plaçons-nous dans le cadre de l'algorithme TD(0) : cela signifie que comme dans le niveau 1 précédent, notre fonction de valeur associe un nombre à chaque état, sous la forme d’un double liste, ou d’un tableau (comme vous préférez l’imaginer) puisque notre jeu n’est pas trop gros. Le principe étant que plus la valeur d’un état est grande, plus l’état est intéressant, découvrons comment actualiser la fonction de valeur : 
Supposons que l’on part de l’état X pour aller dans l’état Y, en obtenant une récompense R. On veut actualiser la valeur associée à l’état X. Pour cela, la recette est la suivante : gardons un peu de l’ancienne valeur, ajoutons-y une dose de la récompense obtenue, et un petit peu de la valeur de Y pour avoir une idée de ce qui nous attend dans le futur. 
Ecrivons un peu mieux cette recette avec la sauce mathématique : 
(même si vous avez fait maths spé, l'interprétation verbale de la formule qui arrive vaut le coup)
-	Appelons 0≤α≤1 le pourcentage d’importance accordé à tout ce qui est nouveau dans l’actualisation (pas l’ancienne valeur de l’état) : c’est notre bien-nommé taux d’apprentissage.
-	Appelons 0≤γ≤1 le rapport dans lequel on prend en considération la valeur de Y pour actualiser la valeur de X dans notre exemple. C’est un paramètre anti-myopie : à 0, on actualise la valeur de X qu’avec la récompense que l’on a obtenue en allant sur Y, et on ne prend donc pas en compte le futur que promet Y. A 1, en revanche, on le prend beaucoup en compte, ainsi que par récurrence les valeurs des états suivants puisque Y les prend aussi en compte dans sa valeur … (tant pis si vous ne comprenez pas parfaitement ça en première lecture)
Cela nous donne donc :

$Valeur(X) = (1- \alpha) \times Valeur(X) + \alpha \times(R + \gamma \times Valeur(Y))$



**Niveau 2**

Plaçons-nous ici aussi dans le cadre d’un jeu pas trop gros, donc notre fonction de valeur de l’algorithme de Q-learning est un tableau prenant en paramètres les états et actions, et sera appelée dans toute la suite et dans les programmes QTable. Bien sûr, toutes les actions ne sont pas toujours possibles depuis chaque état. Enfin, même si informatiquement notre QTable ne sera pas toujours un tableau mais parfois une matrice 3x3 (correspondant aux états) de dictionnaires (actions dans ces états et valeurs), on continuera de l’appeler QTable

Exemple très simple : 

| |Etat #1|Etat #2	|Etat #3|
|-|-|-|-|
|Action #1|	-	|Q(2, 1)|Q(3, 1)|
|Action #2|Q(1, 2)|	Q(2, 2)|Q(3, 2)|
|Action #3|Q(1, 3)| - |Q(3 , 3)|


L'initialisation de la table de Q valeurs (QTable) peut se faire soit de manière aléatoire soit avec que des 0 (pour les actions possibles on n'initalise rien du tout pour les couples $(\text{etat}, \text{action})$ impossibles car la Q-valeur associée ne doit pas exister), par exemple. Sachez que cela n'a aucune importance si vos paramètres sont bien calibrés. Mais quel sont ces paramètres ? Il y en a deux, qui servent à gérer la manière dont on actualise la fonction de valeur. Ils doivent être fixés lorsqu'on entraîne un joueur de manière à converger vers une QTable unique. Le premier paramètre, couramment noté $\alpha \in \left[0,1\right]$ est le taux d'apprentissage. ll détermine à quel point les valeurs de la QTable vont être remplacées lorsqu'on les actualise. Le second, noté $\gamma$, que l'on va appeler le taux de prévision ("discount rate" en anglais). En effet, il caractérise l'importance accordée aux actions futures dans le jeux. A 0, le joueur est myope, à 1, il accorde beaucoup d'importance au futur.

Déterminons à présent que les paramètres sont définis la formule qui régit l'actualisation de la QTable. A chaque fois qu'un joueur effectue une action $a$ depuis un état $s$, on doit actualiser $Q(s, a)$ selon : son état de départ $s$, son état d'arrivée $s'$, et la récompense qu'il a obtenue en effectuant cette action $a$ (dans tous nos jeux, la récompense est associée à un état, donc ici c'est la récompense de l'état d'arrivée qui nous intéresse : $R(s')$ ). Nous avons dit dans la sous-partie précédente que la fonction de valeur (ou QTable) prenait non seulement en compte les récompenses imédiates mais aussi à quel point un état pouvait être intéressant dans le futur. Donc, la nouvelle valeur qu'on va appliquer à notre QTable doit prendre en compte d'un côté $R(s')$, et de l'autre, "à quel point $s'$ est intéressant." Pour cela, on va utiliser la valeur $\max_{a' \in s'} Q(a', s')$ qui utilise notre fonction de valeur. Ce dernier terme va être modulé par $\gamma$ donc, puisque c'est à cela qu'il sert, et ensuite, nous allons actualiser $Q(s, a)$ selon le principe suivant : $\left(1-\alpha\right) \times \text{vieille valeur} + \alpha \times \text{nouvelle valeur}$.
Donc si vous avez bien compris, la formule d'actualisation de la QTable est : 

$Q(s, a) = \left(1-\alpha\right) \times Q(s, a) + \alpha \times \left(R(s') + \gamma \times \max_{a' \in s'} Q(a', s')\right)$


### Synthèse et exemple

Nous avons présenté dans cette partie comment un agent artificiel (un programme informatique quoi) détermine de manière évolutive l'intérêt de chaque action dans chaque état de l'environnement en utilisant une fonction de valeur, et en l’actualisant au fur et à mesure de son entraînement, de la même manière qu'un enfant fait : 

**Niveau 1 très rapide** : L’état « être devant la cheminée » est risqué : il peut mener à effectuer l’action « tendre la main » qui va apporter une récompense très négative, donc une fois que l’enfant aura expérimenté cela une fois (dans le cas ou le cerveau de l’enfant apprend selon la méthode TD(0)), il aura tendance à éviter l’état « être devant la cheminée » à l’avenir, même si cet état peut aussi apporter une récompense intéressante dans le cas de la suite d’action « pas bouger », « pas bouger », « pas bouger », liée au fait d’avoir chaud. Mais comme la sensation de chaleur procure moins de plaisir que le fait de se brûler provoque d’inconfort, globalement la valeur de l’état sera mauvaise … C’est bête, on doit pouvoir faire mieux comme algorithme ! Par exemple, en attribuant plutôt une valeur à un couple $(\text{etat}, \text{action})$, ce qui rendrait l’apprentissage plus précis non ? Tout à fait, c’est l’objet du QLearning et du niveau 2 de cette partie, menant à un tableau de valeurs pour les jeux pas trop gros ressemblant à ça : 

| |Etat #1|Etat #2	|Etat #3|
|-|-|-|-|
|Action #1|	-	|Q(2, 1)|Q(3, 1)|
|Action #2|Q(1, 2)|	Q(2, 2)|Q(3, 2)|
|Action #3|Q(1, 3)| - |Q(3 , 3)|

Au lieu de ça :

|Etat #1	| Etat #2	| Etat #3	| Etat #4 |
|-|-|-|-|
|V(1) | V(2)	| V(3)	| V(4) |

Et ça s’appelle une QTable. (Vous risquez d’en avoir besoin ensuite)

**Niveau 2 très rapide** : Dans l'état "être devant la cheminée", l'action "tendre la main" apporte une récompense très négative, donc la valeur du couple associé diminue. Dans la partie suivante, nous découvrirons comment sont mises en oeuvre différentes stratégies pour que l’agent utilise optimalement sa QTable qu'il essaie de faire converger.


## 3- Dilemme Exploration/Exploitation

Alors que la partie 2 traitait de comment l'agent obtient des informations sur son environnement, nous allons ici voir comment l'agent se comporte dans son environnement grâce aux informations acquises.


###Phases comportementales et stratégies
**Niveau 1**

Les méthodes TD-Learning et Q-Learning permettent donc d'évaluer les actions de l'agent dans son environnement grâce à une fonction de valeur pour pouvoir influencer son comportement futur. Il reste maintenant à définir comment l'agent tient compte de cela pour effectivement influencer son comportement.

Pour cela, on distinguera deux phases différentes dans un stratégie comportementale donnée :

* **L'exploration**: l'agent n'a aucune connaissance, il évolue de manière aléatoire dans son environnement.
* **L'exploitation**: l'agent utilise sa connaissance, il agit de manière optimale, selon la fonction de valeur qu'il cherchera à maximiser.

Phase d'exploration:


![gif_exploration](https://media.giphy.com/media/iDHjpZws3TiLpmv9TM/giphy.gif)



Phase d'exploitation:


![gif_exploitation](https://media.giphy.com/media/jRvfEiBdF7a5RwBmgD/giphy.gif)


Le dilemme est que l'on ne peut pas rester dans une seule de ces deux phases pour l'apprentissage par renforcement. En effet, l'exploitation agira certes de manière optimale mais ne permet pas d'apprendre, tandis que l'exploration de tous les parcours possibles serait trop longue. Il faut donc trouver un bon équilibre entre exploration et exploitation dans l'apprentissage afin que l'agent apprenne vite et bien.



**Niveau 2**

Formellement, une politique est une fonction, qui à un état $s$ et une action $a$ associe une probabilité, celle d'effectuer l'action $a$ dans l'état $s$. Définir la politique consiste donc à établir les probabilités pour chaque état d'effectuer telle ou telle action.
 
Voici deux méthodes pour établir une stratégie dans les algorithmes de TD-Learning et Q-Learning:


#### La méthode $\epsilon$-gloutonne

La méthode $\epsilon$-greedy, ou $\epsilon$-gloutonne, consiste à définir une probabilité $\epsilon$ pour l'agent d'effectuer une action d'exploration, et donc une probabilité $1-\epsilon$ d'effectuer une action d'exploitation (selon la fonction de valeur actuelle). Dans le cas de l'exploration, l'action est choisie de manière aléatoire, indépendamment de la fonction de valeur. Plus $\epsilon$ est proche de 1, plus l'algorithme d'apprentissage est lent, mais la solution sera plus optimale. Avec $\epsilon=1$, la méthode est dite gloutonne.


#### La méthode softmax

Dans certains cas, il est préférable d'éviter les récompenses trop négatives, qui peuvent fausser l'estimation des états. Une solution est d'avoir une probabilité d'effectuer une action a dans l'état s proportionnelle à sa fonction de valeur $Q(s,a)$. Cela permet d'avoir le plus souvent des actions proches de la meilleure:

$$\pi(s,a)=Q(s,a)/∑_{b\in A}Q(s,b)$$

La méthode « softmax » est une amélioration de cette méthode. Elle permet notamment de gérer les cas où $Q$ est à valeur négative.  Elle choisit l’action $a$ avec une probabilité :

$$\pi(s,a)=exp(Q(s,a)/\tau)/∑_{b\in A}exp(Q(s,b)/\tau))$$

Le facteur $\tau$ s'appelle la température, et permet de modifier la balance exploration/exploitation. En effet, si $\tau$ est très grand, les exponnentielles valent toutes 1, et donc la politique suivra une distribution uniforme, donc un comportement d'exploration. A l'inverse, quand $\tau$ tend vers 0, la méthode devient déterministe et on aura un comportement d'exploitation.



### Comment gérer ces deux phases comportementales pour en tirer le meilleur ?
**Niveau 1**

Dans la nature, un animal commence pendant son enfance par un comportement d'exploration. A sa naissance, il ne sait rien et agit de manière aveugle, il apprend petit à petit par essais/erreur. A l'âge adulte, l'animal sera moins téméraire, et agira de manière optimale en se basant sur son expérience. Une méthode d'application est de s'inspirer de la biologie et du comportement animal : cela consiste à régler les facteurs de manière à avoir plutôt beaucoup d'exploration en début d'apprentissage, suivi de plus en plus d'exploitation en fin d'apprentissage. Cela rend les algorithmes beaucoup plus efficaces. 

**Niveau 2**

Le choix de la politique dépend beaucoup du cas d'application. Un des intérêts de ces méthodes est de pouvoir moduler la balance exploration/exploitation au cours de l'apprentissage, par le biais des facteurs $\epsilon$ et $\tau$.
Nous vous encourageons vivement à les tester vous-mêmes, selon votre problème l'une pourra se révéler bien meilleure ... La plus classique et triviale reste la politique $\epsilon$-greedy mais les deux sont rapides à implémenter une fois que le reste est fait.


In [0]:
###CECI EST UN CODE PERMETTANT DE VOIR LA DIFFERENCE ENTRE UN COMPORTEMENT D'EXPLORATION ET UN COMPORTEMENT D'EXPLOITATION 
###SUR UN JOuEUR DEJA BIEN ENTRAINE

###S'IL NE FONCTIONNE PAS DANS LE NOTEBOOK COPIEZ-COLLEZ LE DANS UN FICHIER PYTHON 3.7, C'EST A CAUSE DE TKINTER

###IL EST DERIVE D'UN CODE PLUS COMPLET PERMETTANT D'ENTRAINER ET DE FAIRE JOUER UN JOUEUR SUR N'IMPORTE QUEL JEU 2D
###ET PEUT ETRE ADAPTE A UN PROBLEME PLUS LARGE


#Un joueur évolue sur une grille 2D et doit apprendre à maximiser sa récompense totale en un nombre de coups limités.
#Certaines cases sont recompensées positivement, d'autres négativement et arrêtent la partie.

#Le code complet est long mais très commenté est nous vous encourageons à le lire attentivement après la fin du cours 
#et à le comprendre pour voir la mise en oeuvre pratique de ce qui a été vu. Il est fourni en annexe.



from numpy import math #on importe le module math pour l'infini et sqrt
from random import choice, random
from tkinter import *
import time



class environment():
    """Une instance de cette classe est un environnement, un terrain de jeu, tout ce qui enveloppe le reste."""

    def __init__(self, n, rewarded_states=[], rewards=[], terminal_states=[], forbidden_states=[], start_point=[0, 0], max_plays = 30, move_penalty = -1):

        self.states=[[] for k in range(n)] #la matrice des états de l'environnement = cases.
        #ATTENTION: pour suivre les coordonées Tkinter, c'est une liste de colonnes descendantes. x:gauche->droite ; y:haut->bas

        self.max_plays = max_plays #le nombre de coups (actions) maximum autorisés par partie
        self.move_penalty = move_penalty #
        self.len=n #pratique pour aprèsla récompense fixe attribuée pour chaque action

        #le choix de créer une liste d'états récompensés et une liste des récompenses
        #vient du fait qu'en général il y a peu d'état récompensés et donc
        #il serait lourd d'avoir juste une grosse matrice de récompense quasi-nulle


        #création des états (objets)
        for x in range(n):
            for y in range(n):
                reward=0
                is_terminal=False
                if [x, y] in rewarded_states:
                    indice=rewarded_states.index([x, y])
                    reward=rewards[indice]
                if [x, y] in terminal_states:
                    is_terminal=True
                self.states[x]+=[state(x, y, reward, is_terminal)]

        #rensignement de l'état de départ de l'environnement
        self.start = self.states[start_point[0]][start_point[1]] #état de départ

        #éliminations des actions possibles depuis chaque état selon position (bord) ou états interdits adjacents
        for x in range(n):
            for y in range(n):
                if x==0 or [x-1, y] in forbidden_states:
                    self.states[x][y].possible_actions.remove('Left')
                elif x==n-1 or [x+1, y] in forbidden_states:
                    self.states[x][y].possible_actions.remove('Right')
                if y==0 or [x, y-1] in forbidden_states:
                    self.states[x][y].possible_actions.remove('Up')
                elif y==n-1 or [x, y+1] in forbidden_states:
                    self.states[x][y].possible_actions.remove('Down')

        #initialisation de l'interface graphique
        def color(self, s): #s c'est un state
            return 'red' if s.reward<0 else 'yellow' if s.reward>0 else 'black' if [s.x, s.y] in forbidden_states else 'blue' if s == self.start else None

        #on a a besoin pour après, pas seulement en local
        self.size = int(600/n)*n
        self.step = self.size / n

        self.root = Tk()
        self.root.title('Jeu 2D interactif')

        self.canvas = Canvas(self.root, height=self.size+1+13, width=self.size+1) #le +1 c'est pour les lignes droite et bas
        #self.canvas.create_text(1, self.size+1,anchor='nw', text='Nombre maximal de coups : {} ; Penalité de mouvement : {}'.format(self.max_plays, self.move_penalty))

        #on représente les états par des carrés adéquats sur une grille
        for i in range(n):
            for j in range(n):
                s=self.states[i][j]
                self.canvas.create_rectangle(2+i*self.step, 2+j*self.step, 2+(i+1)*self.step, 2+(j+1)*self.step, fill = color(self, s), width=1, tag = '%d_%d'%(i,j))
                if s.reward!=0:
                    self.canvas.create_text(2+(i+0.5)*self.step, 2+(j+0.5)*self.step, text='{}'.format(s.reward), anchor='center')

        self.canvas.pack(side='left')

        #dans un canvas tkinter, les coordonnées dessinables vont de 2 à n+1 où n est une dimension

#cette classe est aussi utile qu'un dictionnaire qui aurait les mêmes attributs, mais c'est sympa comme ça
class state():
    """Une instance de cette classe est un état de l'environnement, elle est donc générée automatiquement par un environnement
     Un objet état contient toutes les informations nécessaire relatives à lui-même : position sur la grille,
    actions possibles, récompense, terminalité."""

    def __init__(self, x, y, reward, is_terminal):
        self.x=x
        self.y=y
        self.reward=reward
        self.is_terminal=is_terminal
        self.possible_actions=['Left', 'Right', 'Up', 'Down']


#Un petit environnement sympathique
#E=environment(n=7, rewarded_states=[[0,0],[1,0],[6,0],[1,1],[5,1],[4,2],[4,6],[5,6]],
                #terminal_states=[[1,0],[6,0],[1,1],[4,2]], rewards=[100, -1000, -1000, -1000, 10, -1000, 2, 2], start_point=[1, 5])

E=environment(n=4, rewarded_states=[[2, 0], [1, 1]], terminal_states=[[2, 0], [1, 1]], rewards=[100, -100], start_point=[0, 3])




class game():
    """Une instance de cette classe simule une partie du jeu.
Dépend donc d'un objet environnement. Et d'un objet joueur capable de jouer beaucoup de parties.
Dispose de méthode permettant de faire avancer la partie en jouant selon diverses stratégies possibles."""

    def __init__(self, environnement, player, epsilon, tau):
        self.state = environnement.start #on commence la partie sur l'état de départ de l'environnement
        self.previous_state=None
        self.score = 0 #le score cumulatif qui va être réalisé sur la partie
        self.is_over = False #est-ce que la partie est finie
        self.deltamax = 0 #la valeur abs de la correction maximale effectuée sur la QTable du superobjet joueur pendant la partie

        self.P = player
        self.e = epsilon
        self.tau = tau
        self.env = environnement

        self.movements = 0 #le nombre de coups joués sur cette partie. Eh oui, l'environnement limite le nombre de coups jouables par partie.


    def play(self, learning=True, method='e_greedy'): #epsilon-greedy method or softmax or optimal (just play one of the best actions)
        """Effectue une action en actualisant la QTable du joueur et le deltamax ou pas (selon la valeur de learning), et en actualisant le nombre de mouvements, le score et la terminalité."""

        if method=='e_greedy': #la méthode epsilon-greedy
            if random()<self.e:
                order = choice(self.state.possible_actions)
            else:
                order=choice(self.best_actions())

        elif method=='softmax': #la méthode softmax
            QTotal = sum([exp(self.P.QTable[self.state.x][self.state.y][action]/self.tau) for action in self.state.possible_actions])
            if QTotal == 0:
                order = choice(self.state.possible_actions)
            else:
                Proba={} #dico de probas
                for action in self.state.possible_actions:
                    Proba[action]=exp( self.P.QTable[self.state.x][self.state.y][action] / self.tau ) / QTotal
                alea=random() #entre 0 et 1
                if alea<Proba['Left']:
                    order = 'Left'
                elif alea<Proba['Left']+Proba['Right']:
                    order = 'Right'
                elif alea<Proba['Left']+Proba['Right']+Proba['Up']:
                    order = 'Up'
                else:
                    order = 'Down'
                #pas de risque d'action impossible

        elif method=='optimal': #action optimale
            order=choice(self.best_actions())


        #maintenant on gère ce qu'il y a à gérer pour exécuter l'ordre de jeu donné par la méthode choisie
        self.previous_state = self.state
        self.move(order)
        self.movements += 1
        self.score+=self.state.reward + self.env.move_penalty

        if learning: #update de la QTable si learning==True
            old_value = self.P.QTable[self.previous_state.x][self.previous_state.y][order]
            self.P.update_QTable(self.previous_state, order, self.state, self.state.reward + self.env.move_penalty)
            correction = abs(old_value - self.P.QTable[self.previous_state.x][self.previous_state.y][order])
            if correction > self.deltamax :
                self.deltamax = correction

        if self.state.is_terminal == True or self.movements==self.env.max_plays: #on vérifie qu'on a pas fini la partie
            self.is_over = True


    def best_actions(self):
        liste=[]
        Q=self.P.QTable[self.state.x][self.state.y]
        for action in self.state.possible_actions:
            if Q[action] == max(Q.values()): #tout fonctionne parce que les seules clés de Q sont des actions possibles depuis l'état
                liste+=[action]
        return liste


    def move(self, order):
        if order=='Left':
            self.state=self.env.states[self.state.x-1][self.state.y]
        elif order=='Right':
            self.state=self.env.states[self.state.x+1][self.state.y]
        elif order=='Up':
            self.state=self.env.states[self.state.x][self.state.y-1]
        elif order=='Down':
            self.state=self.env.states[self.state.x][self.state.y+1]



class player():
    """Un objet de cette classe va être un superobjet qui contiendra des informations
sur un entraînement réalisé avec une série de parties. Il représente donc un joueur virtuel qui apprend en jouant des parties successives.
La méthode train() va instancier
des objets game (= va créer des parties) et les faire jouer jusqu'à un
état terminal (=va jouer les parties jusqu'au bout) jusqu'à atteindre les critères de convergence.
De là, on pourra accéder à la QTable obtenue suite à cet entraînement, et faire
jouer des parties sans la modifier pour observer le comportement en régime établi,
ou reprendre l'entraînement.
C'est un tel objet qui va être repésenté graphiquement, car il dépend d'un environnement
qu'on pourra représenter, et contient des games qui pourront être représentés
successivement, ainsi qu'une QTable évolutive."""

    def __init__(self, env = E, alpha = 0.75, gamma = 0.9):
        self.env = env #le terrain de jeu

        #la partie
        self.G = game(self.env, self, 1, 20)#juste pour ne pas qu'il y ait d'erreur ; cette partie ne sera jamais jouée et sera écrasée

        #la QTable du joueur , prenant en paramètre les coordonnées puis l'action
        self.QTable=[[{action:0 for action in env.states[x][y].possible_actions} for y in range(self.env.len)]for x in range(self.env.len)]
        #grosse pythonerie, prendre le temps de lire. Et on initialise des Q valeurs que pour les actions possibles à chaque état

        #notre joueur doit avoir un QTable limite unique (dépend de l'environnement et des paramètres alpha et gamma) vers laquelle on tend en l'entraînant
        self.a = alpha
        self.g = gamma

        self.train_nb = 0 #nombre de parties jouées

        self.tempo=True #pour savoir si on affiche les parties lentement pour voir les actions ou si on les joue très vite

        try:
            self.env.canvas.delete('arrow')#enlever les éventuelles flèches d'un précédent joueur
        except:
            pass


    def new_game(self, epsilon, tau):
        self.G = game(self.env, self, epsilon, tau)


    def train(self, N = 200, stop = True , threshold = 0.001, train_type = 'inverseroot', method='e_greedy', epsilon=0.1, tau=20): #on train sur plusieurs parties en explorant d'une certaine façon
        """Fait jouer des joueurs successifs et arrête l'entraînement soit quand le deltamax de la dernière partie jouée est inférieur au seuil (si stop est true)
    soit après avoir fait jouer N joueurs. La méthode renseigne comment vont être jouées les parties, et le type d'entraînement permet un raffinement de la
    manière dont vont être générée les taux d'exploration des parties si on utilise epsilon-greedy. Par exemple, on peut sélectionner une décroissance linéaire."""

        self.new_game(1, tau)#le premier joueur est entièrement aléatoire si on utilise la méthode epsilon-greedy, et est normal avec softmax
        deltamax = math.inf

        while (stop and deltamax>threshold and self.train_nb<N) or (stop==False and self.train_nb<N):
            #on joue des parties

            #position du joueur sur le canvas
            self.graphic_update()

            #pour éventuellement jouer les parties lentement
            if self.tempo:time.sleep(0.05)

            while self.G.is_over==False:
                #on joue un coup
                self.G.play(method=method)

                #actualisation des informations visuelles
                self.show_arrows()
                self.graphic_update()

                if self.tempo:time.sleep(0.05)

            if self.tempo:time.sleep(0.5)

            #on en récupère le deltamax
            deltamax=self.G.deltamax

            #on incrémente le compteur de parties jouées
            self.train_nb+=1

            #nouvelle partie !
            if train_type=='linear':
                self.new_game(1-self.train_nb/N, tau)
            elif train_type=='constant':
                self.new_game(epsilon, tau)
            elif train_type=='exponential':
                self.new_game(math.exp(-self.train_nb), tau)
            elif train_type=='quadratic':
                self.new_game((1-self.train_nb/N)**2, tau)
            elif train_type=='inverseroot':
                self.new_game(1/math.sqrt(self.train_nb + 1), tau)

        #notre QTable a été entraînée !
        self.env.canvas.delete('playerpos')
        print(self.train_nb)



    def switch_tempo(self): #methode pour interaction via IHM
        self.tempo = not self.tempo


    def play(self, epsilon, learning=False, method='e_greedy', tau = 20):
        #joue une partie, par défaut optimale, sans update la QTable
        self.new_game(epsilon, tau)#balec si learning==False
        self.graphic_update()
        time.sleep(0.1)
        while self.G.is_over==False:
            self.G.play(learning, method)
            self.graphic_update()
            time.sleep(0.1)
        self.env.canvas.delete('playerpos')

        return self.G.score


    def update_QTable(self, previous_state, action, state, reward):
        self.QTable[previous_state.x][previous_state.y][action] = (1-self.a)*self.QTable[previous_state.x][previous_state.y][action] + self.a *(reward + self.g * max(list(self.QTable[state.x][state.y].values())))

    def graphic_update(self):
        self.env.canvas.delete('playerpos')
        self.env.canvas.create_oval(self.G.state.x*self.env.step -10 + self.env.step/2,
                                            (self.G.state.y)*self.env.step-10 + self.env.step/2,
                                            self.G.state.x*self.env.step+10 + self.env.step/2,
                                            (self.G.state.y)*self.env.step+10 + self.env.step/2,
                                            fill = 'red', tag='playerpos')
        self.env.canvas.pack()
        self.env.root.update_idletasks()
        self.env.root.update()


    def show_QTable(self): #à optimiser comme pour les flèches, là c'est giga lourd
        self.env.canvas.delete('texte')
        for i in range(self.env.len):
            for j in range(self.env.len):
                s=self.env.states[i][j]
                if 'Left' in s.possible_actions:
                    self.env.canvas.create_text(2+i*self.env.step, 2+(j+0.5)*self.env.step, text=str(int(self.QTable[i][j]['Left'])), anchor='w', tag='texte')
                if 'Right' in s.possible_actions:
                    self.env.canvas.create_text(2+(i+1)*self.env.step, 2+(j+0.5)*self.env.step, text=str(int(self.QTable[i][j]['Right'])), anchor='e', tag='texte')
                if 'Up' in s.possible_actions:
                    self.env.canvas.create_text(2+(i+0.5)*self.env.step, 2+j*self.env.step, text=str(int(self.QTable[i][j]['Up'])), anchor='n', tag='texte')
                if 'Down' in s.possible_actions:
                    self.env.canvas.create_text(2+(i+0.5)*self.env.step, 2+(j+1)*self.env.step, text=str(int(self.QTable[i][j]['Down'])), anchor='s', tag='texte')
        return

    def show_arrows(self):
        i, j = self.G.state.x, self.G.state.y
        self.env.canvas.delete('flèche_%d_%d'%(i, j))
        if 'Left' in self.G.best_actions():
            self.env.canvas.create_line(2+(i+0.25)*self.env.step, 2+(j+0.5)*self.env.step,2+(i-0.25)*self.env.step, 2+(j+0.5)*self.env.step, arrow='last', tag=('flèche_%d_%d'%(i, j), 'arrow'))
        if 'Right' in self.G.best_actions():
            self.env.canvas.create_line(2+(i+0.75)*self.env.step, 2+(j+0.5)*self.env.step,2+(i+1.25)*self.env.step, 2+(j+0.5)*self.env.step, arrow='last', tag=('flèche_%d_%d'%(i, j), 'arrow'))
        if 'Up' in self.G.best_actions():
            self.env.canvas.create_line(2+(i+0.5)*self.env.step, 2+(j+0.25)*self.env.step,2+(i+0.5)*self.env.step, 2+(j-0.25)*self.env.step, arrow='last', tag=('flèche_%d_%d'%(i, j), 'arrow'))
        if 'Down' in self.G.best_actions():
            self.env.canvas.create_line(2+(i+0.5)*self.env.step, 2+(j+0.75)*self.env.step,2+(i+0.5)*self.env.step, 2+(j+1.25)*self.env.step, arrow='last', tag=('flèche_%d_%d'%(i, j), 'arrow'))
        return

player1=player()

def new_player():
    global player1
    player1=player()
    return
def train_player1():
    global player1
    player1.train()
    return
def switch_tempo_player1():
    global player1
    player1.switch_tempo()
    return
def play_player1():
    global player1
    player1.play(scale_epsilon.get())
    return



frame = Frame(E.root, height = E.size+14, width = 100)
frame.pack(side='right')

switch_tempo_player1()
train_player1()
switch_tempo_player1()

scale_epsilon = Scale(frame, orient='horizontal', from_=0, to=1, resolution=0.1, label='Régler epsilon:')
scale_epsilon.pack()

button_play=Button(frame, command=play_player1, text='Jouer une partie')
button_play.pack()


E.root.mainloop()



TclError: ignored

### Courbe de convergence selon epsilon et tau ## 



Voici une étude de l'efficacité des deux méthodes un jeu 2D 7x7 que vous trouverez en annexe, selon les paramètres. Il est représenté ci-dessous. Le but est de faire le meilleur score en 30 coups, sachant que les cases rouges sont mortelles et très punitives en terme de récompense.
Ce jeu est intéressant dans le sens où si le joueur explore trop peu, il se laissera piéger par la facilité de la récompense des 2 à droite, mais ensuite il découvrira le 100 en haut à gauche, plus difficile d'accès.


![jeu7x7](https://image.noelshack.com/fichiers/2019/23/3/1559763588-jeu-7x7.png)

![egreedy](https://image.noelshack.com/fichiers/2019/23/3/1559763578-e-greedy-100-joueurs-annote.png)

![softmax](https://image.noelshack.com/fichiers/2019/23/3/1559763583-softmax-100-joueurs-annote.png)


Il est important de garder à l'esprit que ces méthodes sont basiques et vous en trouverez de plus complexes en explorant le code. Cependant, cela permet de voir que plus l'exploration est importante dans la stratégie comportementale, mieux le joueur entraîné saura jouer mais que l'entraînement sera plus long. Ici la méthode softmax fournit de bons résultats.

## 4 - Exemples

##Définir un environnement


Dans cette partie nous allons pouvoir nous entrainer à mettre en œuvre les principes théoriques vues précédemment. Pour cela nous allons commencer avec un petit exercice visant à définir un environnement pour notre agent. Précédemment on a appris que l’outil utilisé pour représenter l’environnement était les graphes de Markov. Pour implémenter un environnement on commence par définir une classe *environment* :

def __init__(self, n, rewarded_states=[], rewards=[], terminal_states=[], forbidden_states=[], start_point=[0, 0], max_plays = 30, move_penalty = -1):

Ensuite on n'a plus qu’a définir les valeurs des différents paramètres :



Par exemple pour représenter cette grille on définit les paramètres suivants (les coordonnées sont prises par rapport à en haut à gauche pour suivre les coordonées de tkinter):

![Matrice Jeu ](https://media.giphy.com/media/Ss0k78Aos1uiCE6aX9/giphy.gif)

**N**= 4

**rewarded_states**= [ [1,1] , [2,0] ]

**rewards**= [ -100 , 100 ]

**terminal_states** = [ [1,1] ]

On a alors la commande: 

E= *environment* ( n=4, rewarded_states= [ [1,1] , [2,0] ] , rewards= [ -100 , 100 ] , terminal_states = [ [1,1] ] )

###Exercice:

A vous de compléter la commande pour définir l'environnement suivant:

On rappelle que la classe *environment* est définie de la manière suivante: 

*def init(self, n, rewarded_states=[], rewards=[], terminal_states=[], forbidden_states=[], start_point=[0, 0], max_plays = 30, move_penalty = -1)*

![Mat_ex](https://media.giphy.com/media/iD6jZD6Z8ryIOekyZv/giphy.gif)


In [0]:
### A COMPLETER

E = 

### CODE AFFICHAGE: ne pas toucher


###Correction:

In [0]:
### A COMPLETER

E = environnement(n=4, rewarded_states=[[1,1],[0,0],[2,0]] ,rewards=[-100,50,100] ,terminal_states=[[1,1]],forbidden_states=[[1,2],[2,2]] )

## Jeu des allumettes

**Principe du jeu:**
Le jeu se joue à deux, avec un certain nombre d'allumettes, disons 12. Chacun leur tour, les joueurs peuvent prendre 1, 2 ou 3 allumettes. Celui qui prend la dernière allumette perd.

Il existe une méthode optimale permettant au premier joueur de gagner à tous les coups. Il suffit de toujours laisser l'adversaire avec un nombre d'allumettes congru à 1 modulo 4 (1, 5, 9, ...). Voyons si l'apprentissage par renforcement permet de retrouver cette méthode optimale, en utilisant la méthode du TD-Learning.

###Exercice:

Le but de cet exercice est de vous initier à l'implémentation d'une fonction de valeur.

In [0]:
###### Manque les consignes de l'exercice

###Correction:

Tout d'abord, définissons une classe permettant de jouer une partie:

In [0]:
import math as m
import random


# Classe représentant une partie
class StickGame(object):

    def __init__(self, nb, player1, player2):
        self.original_nb = nb  # nombre d'allumettes au départ
        self.nb = nb  # nombre d'allumettes restantes
        self.history = []  # historique de la partie
        self.player1 = player1  # joueur1
        self.player2 = player2  # joueur2
        self.alpha = 0.1        # facteur d'apprentissage

    # Fonction représentant un tour
    def play_action(self):
        action1 = self.player1.action(self.nb)  # action du joueur 1
        nb_before_action = self.nb  # nombre d'allumettes au début du tour
        int_nb = self.nb - action1  # nombre d'allumettes après l'action 1

        if int_nb <= 0:  # Défaite du joueur 1
            print('Player2 wins')
            self.history.append((nb_before_action, 0, 0))
            return 0

        if self.player2.is_human:  # Affichage des allumettes si le joueur 2 est humain
            self.display(int_nb)

        action2 = self.player2.action(int_nb)  # action du joueur 2
        new_nb = int_nb - action2  # nombre d'allumettes après l'action 2
        self.history.append((nb_before_action, int_nb, new_nb))  # ajout du tour à l'historique

        if new_nb <= 0:  # Défaite du joueur 2
            print('Player1 wins')
            return 0
        else:
            return new_nb

    # Afffichage des allumettes
    def display(self, state):
        print("| " * state)

    # Actualise la fonction de valeur V
    def update_V(self):
        for transition in reversed(self.history):  # Parcours à l'envers de l'historique de la partie
            (s, si, sp) = transition  # Un tour s -> si -> sp
            if si == 0:  # Défaite du joueur 1
                V[s] = V[s] + self.alpha * (-1 - V[s])
            elif sp == 0:  # Défaite du joueur 2
                V[s] = V[s] + self.alpha * (1 - V[s])
            else:
                V[s] = V[s] + self.alpha * (V[sp] - V[s])

    # Lancement d'une partie
    def game(self):
        while self.nb > 0:  # Lance un tour tant qu'il reste des allumettes
            self.display(self.nb)
            self.nb = self.play_action()
        self.update_V()  # Actualise V
        self.nb = self.original_nb  # Réinitialise le nombre d'allumettes
        self.history = []  # Réinitialise l'historique

    def run(self):
        self.game()

Définissons maintenant une classe joueur, avec un argument permettant de désigner si le joueur est humain ou non:

In [0]:
# Classe de joueur
class Player(object):

    def __init__(self, is_human):
        self.epsilon = 1
        self.is_human = is_human  # True si humain, False si IA

    def greedy_action(self, state):  # Action d'exploitation de l'IA
        actions = [1, 2, 3]
        vmin = m.inf
        action_min = 1
        for i in actions:
            if state - i > 0 and V[state - i] < vmin:
                vmin = V[state - i]
                action_min = i
        return action_min

    def random_action(self, state):  # Action d'exploration de l'IA
        if state == 1:
            return 1
        elif state <= 3:
            return random.randint(1, state - 1)
        else:
            return random.randint(1, 3)

    def human_action(self):  # Demande d'entrée de l'action humaine
        action = int(input('>:'))
        if action not in [1, 2, 3]:
            action = int(input('>:'))
        return action

    def action(self,
               state):  # Choisis l'action du joueur avec une proba epsilon de choisir l'action d'exploration pour l'IA
        if self.is_human:
            return self.human_action()

        elif random.uniform(0, 1) < self.epsilon:
            return self.random_action(state)

        else:
            return self.greedy_action(state)


Nous pouvons maintenant jouer une partie:

In [0]:
nb = 12
Player_1 = Player(True)
Player_2 = Player(True)
V = {}
for i in range(nb):
    V[i+1] = 0

StickGame(nb, Player_1, Player_2).run()


Essayons maintenant d'apprendre à une IA à jouer. Dans cette phase d'apprentissage, nous ferons affronter deux IA l'une contre l'autre pendant un grand nombre de parties (500), ce qui mettra à jour la fonction de valeur.

L'IA peut jouer de deux manières: par exploration, c'est à dire en choisissant un nombre aléatoire d'allumettes, ou par exploitation, en prenant un nombre d'allumettes qui va minimiser la fonction de valeur pour l'adversaire. Ce taux exploration/exploitation est représenté par $\epsilon$. Pendant l'apprentissage, on commence avec $\epsilon = 1$, c'est-à-dire uniquement de l'exploration, puis on diminue progressivement jusqu'à $\epsilon = 0$, pour n'avoir que de l'exploitation de la part de l'IA.

In [0]:
IA_Player = Player(False)
IA_Player2 = Player(False)
epsilon = 1
learning_nb = 500
LearningGame = StickGame(nb, IA_Player, IA_Player2)

for k in range(learning_nb):
    print('Partie n° ', k+1)
    LearningGame.run()
    epsilon -= 1 / learning_nb
    IA_Player.epsilon = epsilon
    IA_Player2.epsilon = epsilon


A toi de jouer maintenant!

In [0]:
Human_Player = Player(True)
StickGame = StickGame(nb, Human_Player, IA_Player)
for i in range(10):
    StickGame.run()

##Pendule inversé

**Principe:** Le pendule est constitué d'un module pouvant se déplacer horizontalement, et d'une tige rigide fixée au module par son extrémité. Le but est de faire tenir la tige en équilibre au dessus du module, à la verticale.

Pour coder ce pendule inversé, nous utilisons un module python dédié à l'apprentissage par renforcement: gym.

In [0]:
import sys
!{sys.executable} -m pip install gym

In [1]:
import gym
import gym.spaces
import numpy as np
import math
import random
from collections import deque

# Création de l'environnement à partir de la bibliothèque gym
env = gym.make('CartPole-v0')


class CartPole(object):

    def __init__(self, buckets=(1, 1, 6, 3), n_episodes=1000, solved_t=195,
                 min_epsilon=0.1, min_alpha=0.1, gamma=0.99):
        self.buckets = buckets              # (position, vitesse, angle, vitesse angulaire)
        self.n_episodes = n_episodes        # nombre d'épisodes d'entrainement
        self.min_alpha = min_alpha
        self.min_epsilon = min_epsilon
        self.gamma = gamma
        self.solved_t = solved_t            # limite de temps pour un épisode

        self.Q_table = np.zeros(self.buckets + (env.action_space.n,))  # Table de la fonction de valeur Q

    def discretize_state(self, state):
        upper_bounds = env.observation_space.high   # Limites de dimension
        lower_bounds = env.observation_space.low

        upper_bounds[1] = 0.5                       # Limites de vitesse et de vitesse angulaire
        upper_bounds[3] = math.radians(50)
        lower_bounds[1] = -0.5
        lower_bounds[3] = -math.radians(50)

        # Discrétisation des dimensions
        width = [upper_bounds[i] - lower_bounds[i] for i in range(len(state))]
        ratios = [(state[i] - lower_bounds[i]) / width[i] for i in range(len(state))]
        bucket_indices = [int(round(ratios[i] * (self.buckets[i] - 1))) for i in range(len(state))]
        bucket_indices = [max(0, min(bucket_indices[i], self.buckets[i] - 1)) for i in range(len(state))]

        return tuple(bucket_indices)

    # Implémentation de la méthode epsilon-gloutonne
    def select_action(self, state, epsilon):
        if random.random() <= epsilon:
            return env.action_space.sample()  # exploration
        else:
            return np.argmax(self.Q_table[state])  # exploitation

    # Choisis un epsilon entre min_epsilon et 1
    def get_epsilon(self, episode_number):
        return max(self.min_epsilon, min(1, 1 - math.log10((episode_number + 1) / 25)))

    # Choisis un alpha entre min_alpha et 1
    def get_alpha(self, episode_number):
        return max(self.min_alpha, min(1, 1 - math.log10((episode_number + 1) / 25)))

    # Actualisation de la table de valeur de Q selon la méthode de Q-Learning
    def update_table(self, old_state, action, reward, new_state, alpha):
        new_state_Q_value = np.max(self.Q_table[new_state])
        self.Q_table[old_state][action] += alpha * (
                    reward + self.gamma * new_state_Q_value - self.Q_table[old_state][action])

    def run(self):
        # Continue les episodes tant que la récompense moyenne de 100 episodes est en dessous de solved_t
        scores = deque(maxlen=100)

        for episode in range(self.n_episodes):
            #Déroulé d'un épisode
            done = False
            alpha = self.get_alpha(episode)
            epsilon = self.get_epsilon(episode)
            episode_reward = 0                      # récompense = temps de l'épisode

            obs = env.reset()                       # réinitialisation de l'environnement
            curr_state = self.discretize_state(obs)

            while not done:
                env.render()
                action = self.select_action(curr_state, epsilon)
                obs, reward, done, info = env.step(action)
                new_state = self.discretize_state(obs)

                self.update_table(curr_state, action, reward, new_state, alpha)
                curr_state = new_state
                episode_reward += reward

            scores.append(episode_reward)
            mean_reward = np.mean(scores)

            # Affichage des épisodes
            if mean_reward > self.solved_t and (episode + 1) >= 100:
                print("Ran {} episodes, solved after {} trials".format(episode + 1, episode + 1 - 100))
                return episode + 1 - 100
            elif (episode + 1) % 50 == 0 and (episode + 1) >= 100:
                print("Episode number: {}, mean reward over past 100 episodes is {}".format(episode + 1, mean_reward))
            else:
                print("Episode {}, reward {}".format(episode + 1, episode_reward))


if __name__ == "__main__":
    cartpole = CartPole()
    cartpole.run()



NoSuchDisplayException: ignored